# **Usando Python e dados abertos no Google Colab**

Bruno Gimenes Delphim

[*'Discente do curso de Biblioteconomia e Ciência da Informação - Campus USP-RP*](https://github.com/bgd94)

Neste notebook do Google Colab, eu vou mostrar como utilizar Python para acessar dados abertos do governo.

---

## Python

## catalogos-dados-brasil (**tableschema & datapackage**)
Conforme https://github.com/dadosgovbr/catalogos-dados-brasil/blob/master/scripts/uso/como-usar-com-o-pandas.ipynb

### Utilizando o terminal/prompt de comando para instalar pacotes Python externos

In [145]:
pip install datapackage tableschema-pandas | pip install plotly plotly_express

#### Imprimindo versões instaladas

In [146]:
pip freeze | grep -e datapackage -e tableschema -e plotly

datapackage==1.15.4
plotly==5.15.0
plotly-express==0.4.1
tableschema==1.20.11
tableschema-pandas==1.1.0


### Lendo o pacote de dados (**plotly**)

In [144]:
import pandas as pd

# Para trabalhar com Frictionless Data – frictionlessdata.io
from tableschema import Storage
from datapackage import Package

# Para visualização
import plotly_express as px
import plotly as py, plotly.graph_objects as go


# Tentar gravar no Panda e imprimir "true" se for bem-sucedido
output = "true"
try:
    url = 'https://github.com/dadosgovbr/catalogos-dados-brasil/raw/master/datapackage.json'

    # Conectar ao armazenamento Pandas
    storage = Storage.connect('pandas')

    # Carregar o pacote de dados da URL e salvar no armazenamento
    package = Package(url)
    package.save(storage=storage)

    # Se não houver exceções, imprimir "true"


except Exception as e:
    # Se ocorrer uma exceção, imprimir "false" e a mensagem de erro
    output = "false"
    print(e)

output

storage.buckets

type(storage['catalogos'])

storage['solucao']

storage['catalogos'].head()

tipo_solucao = storage['catalogos'].groupby('Solução').count()['URL'].rename('quantidade')
tipo_solucao

px.bar(
    pd.DataFrame(tipo_solucao).reset_index(),
    x = 'Solução',
    y = 'quantidade',
    color = 'Solução',
    color_discrete_sequence = py.colors.qualitative.Set2
)

poder = storage['catalogos'].groupby('Poder').count()['URL'].rename('quantidade')
poder

go.Figure(
    data=go.Pie(
        labels=poder.index,
        values=poder.values,
        hole=.4
    )
).show()

esfera = storage['catalogos'].groupby('Esfera').count()['URL'].rename('quantidade')
esfera

go.Figure(
    data=go.Pie(
        labels=esfera.index,
        values=esfera.values,
        hole=.4
    )
).show()

uf = storage['catalogos'].groupby('UF').count()['URL'].rename('quantidade')
uf

px.bar(
    pd.DataFrame(uf).reset_index(),
    x = 'UF',
    y = 'quantidade',
    color = 'UF',
    color_discrete_sequence = py.colors.qualitative.Set3
)

## excercicio-dados-sp (**frictionless**)


### Utilizando o terminal/prompt de comando para instalar pacotes Python externos

In [147]:
pip install frictionless | pip install plotly plotly_express

#### Imprimindo versões instaladas

In [148]:
pip freeze | grep -e frictionless -e plotly

frictionless==5.16.1
plotly==5.15.0
plotly-express==0.4.1


### Lendo o pacote de dados

In [149]:
import pandas as pd
from google.colab import drive
from frictionless import describe, extract, validate
from pprint import pprint

In [150]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [151]:
def convert_excel_to_csv(path, file, extension):
  content = pd.read_excel(f"{path}/{file}.{extension}")

  content.to_csv(f"{file}.csv", mode = "a", index = None, header = True)

  return f"{file}.csv"

In [152]:
csv_file_path = convert_excel_to_csv("/content/drive/MyDrive/Dataset", "IDESP_ESCOLA_2022", "xlsx")

In [153]:
# Describing Data
resource = describe(csv_file_path)
pprint(resource)

{'name': 'idesp_escola_2022',
 'type': 'table',
 'path': 'IDESP_ESCOLA_2022.csv',
 'scheme': 'file',
 'format': 'csv',
 'mediatype': 'text/csv',
 'encoding': 'utf-8',
 'schema': {'fields': [{'name': 'ANO', 'type': 'integer'},
                       {'name': 'CODIGO_CIE', 'type': 'integer'},
                       {'name': 'CODIGO_INEP', 'type': 'integer'},
                       {'name': 'CODIGO_DIRETORIA', 'type': 'integer'},
                       {'name': 'NOME_DIRETORIA', 'type': 'string'},
                       {'name': 'NOME_ESCOLA', 'type': 'string'},
                       {'name': 'NOME_MUNICIPIO', 'type': 'string'},
                       {'name': 'ANOS_INICIAIS', 'type': 'number'},
                       {'name': 'ANOS_FINAIS', 'type': 'number'},
                       {'name': 'ENSINO_MEDIO', 'type': 'number'}]}}


In [161]:
# Extracting Data
# Add/adjust metadata if necessary (field_missing_values, resource.schema.foreign_keys, .yaml)

rows = extract(csv_file_path)
#pprint(rows)

first_or_default = list(rows.values())[0]
print(first_or_default)

[{'ANO': 2022, 'CODIGO_CIE': 12, 'CODIGO_INEP': 35000012, 'CODIGO_DIRETORIA': 10101, 'NOME_DIRETORIA': 'NORTE 1', 'NOME_ESCOLA': 'AYRES DE MOURA PROFESSOR', 'NOME_MUNICIPIO': 'SAO PAULO', 'ANOS_INICIAIS': None, 'ANOS_FINAIS': Decimal('4.25'), 'ENSINO_MEDIO': None}, {'ANO': 2022, 'CODIGO_CIE': 24, 'CODIGO_INEP': 35000024, 'CODIGO_DIRETORIA': 10101, 'NOME_DIRETORIA': 'NORTE 1', 'NOME_ESCOLA': 'GAVIAO PEIXOTO BRIGADEIRO', 'NOME_MUNICIPIO': 'SAO PAULO', 'ANOS_INICIAIS': Decimal('4.0'), 'ANOS_FINAIS': Decimal('2.53'), 'ENSINO_MEDIO': Decimal('0.95')}, {'ANO': 2022, 'CODIGO_CIE': 36, 'CODIGO_INEP': 35000036, 'CODIGO_DIRETORIA': 10101, 'NOME_DIRETORIA': 'NORTE 1', 'NOME_ESCOLA': 'JOAO SOLIMEO', 'NOME_MUNICIPIO': 'SAO PAULO', 'ANOS_INICIAIS': None, 'ANOS_FINAIS': Decimal('2.33'), 'ENSINO_MEDIO': Decimal('1.09')}, {'ANO': 2022, 'CODIGO_CIE': 48, 'CODIGO_INEP': 35000048, 'CODIGO_DIRETORIA': 10101, 'NOME_DIRETORIA': 'NORTE 1', 'NOME_ESCOLA': 'WALFREDO ARANTES CALDAS PROFESSOR', 'NOME_MUNICIPIO': 

In [ ]:
#Validating Data

report = validate(csv_file_path)
pprint(report)